In [1]:
import pandas as pd

In [5]:
import requests
import time
import random
import csv
import datetime


# --- CẤU HÌNH NGƯỜI DÙNG (BẠN CHỈ CẦN SỬA DÒNG NÀY) ---
# Copy toàn bộ nội dung 'cookie' trong tab Headers dán vào giữa cặp nháy đơn ''
my_cookie = 'ZConsent=timestamp=1760327803197&location=https://zalo.me/pc; __zi=3000.SSZzejyD0jydXQckra00a3BBfxQL71AQV8UbljfP5v1yYAVmsLWKt7wDflZQNHlTCm.1; __zi-legacy=3000.SSZzejyD0jydXQckra00a3BBfxQL71AQV8UbljfP5v1yYAVmsLWKt7wDflZQNHlTCm.1; ozi=2000.SSZzejyD0jydXQckra00a3BBfxQK71AQVOUaljvP5vPwYQkusn8Vatd5gRtQ6XoME3ap.1; ZConsent=timestamp=1760327803197&location=https://zalo.me/pc; _fbp=fb.1.1763434116272.510980907223108041; _ga_RYD7END4JE=GS2.2.s1763434193$o2$g1$t1763434194$j59$l0$h0; _ga_YS1V643LGV=GS2.1.s1763434192$o2$g0$t1763434493$j60$l0$h0; zpsid=Pod3.282972265.65.rxTiwz9KZnwxX58EtbIF7gSd_IpgPhOfu6QvAxKHm9FCIIYvqzTk1RjKZnu; _gcl_au=1.1.1485582632.1763434115; _hjSessionUser_3705269=eyJpZCI6IjA1YTIzZmZjLTJlY2QtNWU4My1hNTM1LTlhNTE1MWJjY2EyMCIsImNyZWF0ZWQiOjE3NjM0MzQ4ODc2MjEsImV4aXN0aW5nIjp0cnVlfQ==; _ga_S4Y4BPVHZV=GS2.1.s1763434942$o1$g1$t1763434943$j59$l0$h0; _ga_C51H66N22P=GS2.2.s1763448660$o2$g0$t1763448660$j60$l0$h0; _ga_4QSSY47VGF=GS2.2.s1763448660$o2$g0$t1763448660$j60$l0$h0; _ga_GP4XVYJ45L=GS2.1.s1763448660$o2$g0$t1763448660$j60$l0$h0; _zlang=vn; _gid=GA1.2.613380716.1764146568; _ga_3EM8ZPYYN3=GS2.2.s1764146568$o11$g0$t1764146568$j60$l0$h0; ZPIDaefdd=s392mzlcc8tu11cjvukyn7sp7; ZPIDff384=6skq4e7rhjb11ol182v2yvs6a; __oam_sid=58b9814978b20bed52a2.1+JHo8J4vqQLsqblaNszktesJRawodOe1DE7+FjhQFQ=; zlanguage=VN; __zactoa_fb8cc3a622f3cbad92e2=MTkzNzAzNTExNTU1NjM0NTg4OQ==; __zapage=1a10aeb2ebf802a65be9; _ga_WSPJQT0ZH1=GS2.1.s1764147323$o2$g0$t1764147334$j49$l0$h0; zoalang=vn; _ga=GA1.2.264635782.1760327782; _ga_NVN38N77J3=GS2.2.s1764147308$o3$g1$t1764147339$j29$l0$h0; _ga_E63JS7SPBL=GS2.1.s1764147308$o3$g1$t1764147340$j28$l0$h0; zoats=afc56d11adeadeb587fa'


def get_broadcast_history_and_save():
    url = "https://oa.zalo.me/manage/broadcast/history"
   
    # Headers chuẩn theo trình duyệt Chrome 142 của bạn
    headers = {
        'authority': 'oa.zalo.me',
        'accept': '*/*',
        'accept-language': 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
        'cookie': my_cookie,
        'referer': 'https://oa.zalo.me/manage/broadcast/compose',
        'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    }


    all_data = []
    page = 1
   
    print(">>> Bắt đầu lấy dữ liệu...")


    while True:
        print(f"--- Đang tải trang {page} ---")
       
        params = {
            'action': 'getdata',
            'pageIndex': str(page),
            'pageCount': '10', # Giữ nguyên 10 để an toàn
            'fromDate': '',
            'toDate': ''
        }


        try:
            response = requests.get(url, headers=headers, params=params)
           
            # 1. Kiểm tra lỗi Cookie hết hạn
            if "login" in response.url or "<!DOCTYPE html>" in response.text:
                print("❌ LỖI: Cookie đã hết hạn hoặc không đúng!")
                print("👉 Vui lòng F12 lấy lại cookie mới và dán lại vào code.")
                break
           
            # 2. Xử lý dữ liệu JSON
            data = response.json()
            items = data.get('broadcasts', [])
           
            if not items:
                print("✅ Đã hết dữ liệu để lấy.")
                break
               
            # 3. Lọc lấy các trường cần thiết
            for item in items:
                articles = item.get('articles', [])

                title = None
                if isinstance(articles, list) and len(articles) > 0:
                    title = articles[0].get('title')

                row = {
                    'ID': item.get('broadcastId'),
                    'Tiêu đề' : title,
                    'Trạng thái': item.get('statusInfo'), # Thường là 1, 2, 3 (cần map nếu muốn hiện chữ)
                    'Lượt xem': item.get('numberReceive', 0),
                    'Lượt gửi': item.get('numberSend', 0),
                    'Lượt click': item.get('numberView', 0), # Nếu có
                    'Ngày tạo': item.get('time')
                }
                all_data.append(row)
                print(f"   + Đã lấy: {row['Tiêu đề']}")
           
            # 4. Ngủ ngẫu nhiên để tránh bị chặn (QUAN TRỌNG)
            sleep_time = random.uniform(2, 5)
            time.sleep(sleep_time)
           
            page += 1
           
        except Exception as e:
            print(f"❌ Có lỗi xảy ra: {e}")
            break


    # --- LƯU RA FILE CSV (Mở được bằng Excel) ---
    if all_data:
        filename = f"zalo_broadcast_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
       
        # 'utf-8-sig' giúp Excel hiển thị đúng tiếng Việt
        with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=all_data[0].keys())
            writer.writeheader()
            writer.writerows(all_data)
           
        print(f"\n🎉 HOÀN TẤT! Đã lưu {len(all_data)} dòng vào file: {filename}")
    else:
        print("\n⚠️ Không lấy được dữ liệu nào.")


if __name__ == "__main__":
    if my_cookie == 'DÁN_COOKIE_CỦA_BẠN_VÀO_ĐÂY':
        print("❌ BẠN CHƯA DÁN COOKIE!")
        print("👉 Hãy sửa dòng số 8 trong code: my_cookie = 'copy_cookie_paste_vao_day'")
    else:
        get_broadcast_history_and_save()


>>> Bắt đầu lấy dữ liệu...
--- Đang tải trang 1 ---
   + Đã lấy: Hhhhhhhhhhhhhhhhhhh
   + Đã lấy: BẬT VIBE CHILL – TRI ÂN “THẦY IU” MỪNG 20/11
   + Đã lấy: HÒA VÀO ĐÊM HALLOWEEN BUNG NÓC
   + Đã lấy: KHUẤY ĐẢO CÙNG HỘI CHỊ EM VỚI HER-ICANE NIGHT
   + Đã lấy: DZÔKTOBERFEST - Enjoy Oktoberfest kiểu Việt
   + Đã lấy: VẼ ĐÈN XINH XINH – TRUNG THU BLINK BLINK
   + Đã lấy: CUỐI TUẦN XẢ STRESS CÙNG SIPPIN’ SUNDAY
   + Đã lấy: CHAY O’CLOCK – NHẬU CHAY VẪN CHÁYYY 🌱🍻
   + Đã lấy: TỰ HÀO HƯỞNG VỊ TỰ DO – MỪNG ĐẠI LỄ 2/9 TẠI BA GÁC
   + Đã lấy: Không thể bỏ lỡ SUMMÊ-LY SPRITZ
--- Đang tải trang 2 ---
   + Đã lấy: 🍻 Khám phá hương vị Bỉ cùng Belgo Tap Takeover
   + Đã lấy: 🍻Thưởng thức vị mùa hè mê ly cùng TAP TAKEOVER
   + Đã lấy: HỘI CẠ CỨNG ĐỪNG BỎ LỠ “BESTIE NIGHT” TỐI THỨ BẢYYY
   + Đã lấy: ĐỪNG BỎ LỠ LADIES NIGHT TỐI THỨ SÁUUU
   + Đã lấy: CHẤN ĐỘNG VỚI DEAL $1 MỖI NGÀY TẠI TẤT CẢ CHI NHÁNH!!!
   + Đã lấy: 🧡CHÚC MỪNG NGÀY GIA ĐÌNH VIỆT NAM 28/06
   + Đã lấy: ✈️ LÊN NÓC CUỐI TUẦN TẠI BA GÁC NA

In [6]:
import requests

url = "https://www.facebook.com/data/manifest/?is_workplace_mobile_pwa_dogfooding=0"

cookies = {
    "c_user": "100014844849106",
    "xs": "32%3AmYDg6BNBRghsVw%3A2%3A1754712253%3A-1%3A-1%3A%3AAcwb8gMWCo5EzKf_kWBKAyBFvjmSuvYNPvADh3_R0pDA",
    "datr": "cciWaDlYfs-BkbyxAKOsnoLA",
    "sb": "cciWaP4G179hzdP_pdHqdI0i",
    "fr": "1knBfRW2XvuS2EKvL.AWcjPpShz6fpjzL4hfQCBvIK4BZxkIuVDFoTlyAW5q3Pgpfn1lk.BpKCmc..AAA.0.0.BpKDU1.AWdrxuXRaP5YfhOQabhR0CcEZNk",
    "presence": "C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1764241901152%2C%22v%22%3A1%7D",
    "wd": "162x695",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Referer": "https://www.facebook.com/",
    "Accept": "*/*",
}

response = requests.get(url, cookies=cookies, headers=headers)

print(response.status_code)
print(response.text[:500])  # print first 500 chars


200
{"gcm_sender_id":"15057814354","gcm_user_visible_only":true,"edge_side_panel":{"preferred_width":376},"short_name":"Facebook","name":"Facebook","start_url":"\/?ref=homescreenpwa","display":"minimal-ui","background_color":"#FFFFFF","theme_color":"#1877F2","icons":[{"src":"https:\/\/static.xx.fbcdn.net\/rsrc.php\/v4\/y0\/r\/eFZD1KABzRA.png","sizes":"192x192","type":"image\/png"},{"src":"https:\/\/static.xx.fbcdn.net\/rsrc.php\/v4\/yd\/r\/DeNyZD1Vj3q.png","sizes":"512x512","type":"image\/png"}],"la
